<img src="https://www.escoladnc.com.br/wp-content/uploads/2022/06/dnc_formacao_dados_logo_principal_preto-1.svg" alt="drawing" width="300"/>

# Dinâmica: Recomendação com seleção de acervo e ItemKNN

Neste notebook iremos recomendar filmes do gênero `Drama` a partir do histórico de consumo do usuário utilizando o algoritmo ItemKNN. Utilizaremos a implementação da biblioteca [surprise](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans) criada a partir do sklearn.

**Nota**: Para instalar a biblioteca `surprise` descomente a linha abaixo e execute a célula.

In [6]:
# !pip install scikit-surprise

In [7]:
import os
import numpy as np
import pandas as pd
# from google.colab import files

# Carregando o dataset

Nesta dinâmica utilizaremos o `MovieLens`, dataset que contém avaliações de usuários para filmes que foi explorado ao longo do curso. Em particular, carregaremos os seguintes arquivos:

- `ratings.parquet`: avaliações dos usuários para filmes
- `movies.parquet`: metadados dos filmes

## Arquivo de avaliações

Upload file `ratings.parquet`

In [8]:
# %%time
# _ = files.upload() # approx: 1min10s

In [9]:
df_ratings = pd.read_parquet('../../../../data/ratings.parquet')
df_ratings.tail()

,user_id,item_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


## Arquivo de metadados dos itens

Upload file `movies.parquet`

In [10]:
# %%time
# _ = files.upload() # approx: 10s

In [11]:
df_items = pd.read_parquet('../../../../data/movies.parquet')
df_items.set_index('item_id', inplace=True)
df_items.tail()

,title,genres
item_id,,
3948,Meet the Parents (2000),Comedy
3949,Requiem for a Dream (2000),Drama
3950,Tigerland (2000),Drama
3951,Two Family House (2000),Drama
3952,"Contender, The (2000)",Drama|Thriller


# Pré-processamento

Para treinar um algoritmo que recomenda filmes do gênero `Drama` precisamos filtrar somente os itens que possuem este gênero e as avaliações dos usuários para estes itens.

In [12]:
# Filtre os itens que contém o gênero drama na coluna "genres"
genre = 'Drama'
df_items = df_items[df_items['genres'].str.contains(genre)]
df_items.tail()

,title,genres
item_id,,
3946,Get Carter (2000),Action|Drama|Thriller
3949,Requiem for a Dream (2000),Drama
3950,Tigerland (2000),Drama
3951,Two Family House (2000),Drama
3952,"Contender, The (2000)",Drama|Thriller


In [13]:
# Faça um inner join entre o dataframe de avaliações e o dataframe de itens filtrado
df_ratings = df_ratings.merge(df_items, on='item_id', how='inner')
df_ratings.tail()

,user_id,item_id,rating,timestamp,title,genres
354524,5334,127,1,960795494,"Silence of the Palace, The (Saimt el Qusur) (1...",Drama
354525,5334,3382,5,960796159,Song of Freedom (1936),Drama
354526,5675,2703,3,976029116,Broken Vessels (1998),Drama
354527,5780,2845,1,958153068,White Boys (1999),Drama
354528,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western


## Definindo os datasets de treino e validação

Como o ItemKNN é um modelo de parâmetros treináveis, podemos separar o dataset em treino e validação para observar se o treinamento está com uma boa generalização.

Dada a natureza sequencial do consumo de filmes, iremos utilizar o campo `timestamp` para fazer a quebra entre treino e validação: os primeiros `train_size` registros serão utilizados como treino e o restante como teste.

Além disso, a biblioteca `surprise` requer os seguintes nomes de colunas:

- `userID`: identificador do usuário
- `itemID`: identificador do item
- `rating`: _feedback_ do usuário

**Dica**: utilize o método [np.split](https://numpy.org/doc/stable/reference/generated/numpy.split.html) para separar os N primeiros registros de um dataframe ordenado para criar os conjuntos de treino e validação.


In [14]:
train_size = 0.8

# Ordene por timestamp
df_ratings = df_ratings.sort_values(by='timestamp', ascending=True)

# Defina os conjuntos de treinamento e validação
df_train_set, df_valid_set = np.split(df_ratings, [ int(train_size*df_ratings.shape[0]) ])

# Renomeie as colunas user_id e item_id para userID e itemID, respectivamente
df_train_set = df_train_set.rename({'user_id': 'userID', 'item_id': 'itemID'}, axis=1)
df_valid_set = df_valid_set.rename({'user_id': 'userID', 'item_id': 'itemID'}, axis=1)

print ('Train size: ', df_train_set.shape)
print ('Valid size: ', df_valid_set.shape)

Train size:  (283623, 6)
Valid size:  (70906, 6)


In [15]:
df_train_set

,userID,itemID,rating,timestamp,title,genres
197924,6040,858,4,956703932,"Godfather, The (1972)",Action|Crime|Drama
66982,6040,593,5,956703954,"Silence of the Lambs, The (1991)",Drama|Thriller
17212,6040,1961,4,956703977,Rain Man (1988),Drama
240799,6040,2019,5,956703977,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama
288585,6040,1419,3,956704056,Walkabout (1971),Drama
...,...,...,...,...,...,...
202620,610,2000,2,975861343,Lethal Weapon (1987),Action|Comedy|Crime|Drama
256745,610,2352,3,975861367,"Big Chill, The (1983)",Comedy|Drama
90071,973,1683,2,975861377,"Wings of the Dove, The (1997)",Drama|Romance|Thriller
202258,973,1633,4,975861377,Ulee's Gold (1997),Drama


## Criando um dataset para a biblioteca Surprise

Algumas bibliotecas de sistemas de recomendação possuem classes específicas para trabalhar com os datasets. Desta forma, é preciso modificar os dados originais (em `pandas Dataframes` para as classes da biblioteca `surprise`.

In [16]:
from surprise import Dataset, Reader
def convert_train_valid_sets(df_train_set:pd.DataFrame, df_valid_set:pd.DataFrame):
  reader = Reader(rating_scale=(1, 5))
  # The columns must correspond to user id, item id and ratings (in that order).
  train_set = (
      Dataset
      .load_from_df(df_train_set[['userID', 'itemID', 'rating']], reader)
      .build_full_trainset()
  )

  valid_set = (
      Dataset
      .load_from_df(df_valid_set[['userID', 'itemID', 'rating']], reader)
      .build_full_trainset()
      .build_testset()
  )

  return train_set, valid_set

train_set, valid_set = convert_train_valid_sets(df_train_set, df_valid_set)

In [17]:
train_set

# Treinando o modelo

Para este exemplo prático iremos utilizar o **KNNWithMeans**, cuja fórmula de predição é dada abaixo:

$$\hat{r}_{ui} = b_i + \frac{\sum_{j \in N_u^k(i)} sim(i,j) \cdot (r_{uj} - b_j)}{\sum_{j \in N_u^k(i)} sim(i,j)}$$

Os hiperparâmetros escolhidos serão os indicados pela documentação para o dataset do MovieLens. No entanto, para outros datasets é preciso que haja uma busca de hiperparâmetros que seja ideal para os dados em questão. Para ler a definição dos hiperparâmetros, consulte a [documentação](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans).

**Nota**: para ler mais sobre testagem de hiperparâmetros, consulte a biblioteca [HyperOpt](http://hyperopt.github.io/hyperopt/).

In [18]:
from surprise import KNNWithMeans

k = 40
sim_options = {
    "name": "pearson_baseline",
    "user_based": False,  # compute similarities between items
}

# Instancie o modelo a partir da classe KNNWithMeans
model = KNNWithMeans(k=k, sim_options=sim_options, verbose=True)
model

Treinando o ItemKNN

In [19]:
%%time
# Treine o modelo no dataset de treinamento da biblioteca surprise
model.fit(train_set)

Estimating biases using als...


Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
CPU times: user 2.74 s, sys: 61.5 ms, total: 2.8 s
Wall time: 2.79 s


## Predição do conjunto de treino e validação

Uma vez que o modelo esteja treinado, a predição de rating para um par usuário-item pode ser feita a partir do seguinte método:

```python
model.predict(uid, iid)
```

Que retorna um objeto da classe `Prediction` com os seguintes atributos:
- `uid`: ID do usuário passado para o método
- `iid`: ID do item passado para o método
- `est`: estimativa da avaliação para o par uid-iid
- `r_ui`: avaliação original do par uid-iid (se disponível)

In [20]:
prediction = model.predict(uid=5950, iid=3328)
prediction

Prediction(uid=5950, iid=3328, r_ui=None, est=3.3616844387710287, details={'actual_k': 40, 'was_impossible': False})

Nas células abaixo, aplique a função `model.predict` para todos os pares usuário-item nos conjuntos de treino e validação para obter a estimativa da avaliação. Guarde a estimativa em uma coluna chamada **prediction**.

In [21]:
%%time
# Extraia a estimativa de avaliação para todos os pares usuário-item
df_train_set['prediction'] = df_train_set.apply(
    lambda x: model.predict(uid=x['userID'], iid=x['itemID']).est,
    axis=1
)

CPU times: user 36.2 s, sys: 68.6 ms, total: 36.3 s
Wall time: 36.3 s


In [22]:
%%time
# Extraia a estimativa de avaliação para todos os pares usuário-item
df_valid_set['prediction'] = df_valid_set.apply(
    lambda x: model.predict(uid=x['userID'], iid=x['itemID']).est,
    axis=1
)

CPU times: user 4.69 s, sys: 4.04 ms, total: 4.69 s
Wall time: 4.69 s


In [23]:
df_valid_set.tail()

,userID,itemID,rating,timestamp,title,genres,prediction
314581,5950,3328,3,1046369090,Ghost Dog: The Way of the Samurai (1999),Crime|Drama,3.361684
320011,5950,3317,3,1046369439,Wonder Boys (2000),Comedy|Drama,3.676968
33131,5950,3578,4,1046369670,Gladiator (2000),Action|Drama,3.196933
267347,5948,3098,4,1046437932,"Natural, The (1984)",Drama,4.340451
207771,4958,2453,4,1046454260,"Boy Who Could Fly, The (1986)",Drama|Fantasy,3.218370


# Avaliando o modelo

Para avaliar o modelo, utilize a função `rmse` abaixo e compare a métrica nos conjuntos de treinamento e validação.

In [24]:
from sklearn.metrics import mean_squared_error
def root_mean_squared_error(y_true, y_target):
  return np.sqrt(mean_squared_error(y_true, y_target))

# Calcule o RMSE para o conjunto de treinamento
rmse_train = root_mean_squared_error(df_train_set['rating'], df_train_set['prediction'])
print ('RMSE do conjunto de treinamento:', rmse_train)

RMSE do conjunto de treinamento: 0.47183072575412355


In [25]:
# Calcule o RMSE para o conjunto de validação
rmse_valid = root_mean_squared_error(df_valid_set['rating'], df_valid_set['prediction'])
print ('RMSE do conjunto de treinamento:', rmse_valid)

RMSE do conjunto de treinamento: 0.9741044200802675


# Gerando Recomendações

Para finalizar esta dinâmica, experimente recomendar N filmes para diferentes usuários a partir das células abaixo.

In [26]:
user_id = 1875
n = 5

In [27]:
def recommend_n_items(model, user_id, item_ids:np.array, n=20, user_consumed_items:np.array=None):
  df_predictions = pd.DataFrame(columns=['item_id', 'score'])

  if user_consumed_items is not None:
    item_ids = item_ids[~np.isin(item_ids, user_consumed_items)]
  
  for item_id in item_ids:
    prediction = model.predict(uid=user_id, iid=item_id).est
    df_predictions.loc[df_predictions.shape[0]] = [item_id, prediction]
  
  user_predictions = (
      df_predictions
      .sort_values(by='score', ascending=False)
      .head(n)
      .set_index('item_id')
  )
  return user_predictions

item_ids = df_items.index.values
user_consumed_items = df_ratings.query('user_id == @user_id')['item_id'].unique()
recommendations = recommend_n_items(model, user_id, item_ids, n, user_consumed_items)
recommendations.merge(df_items, left_index=True, right_index=True)

,score,title,genres
item_id,,,
989.0,5.0,Schlafes Bruder (Brother of Sleep) (1995),Drama
3522.0,5.0,Sacco and Vanzetti (Sacco e Vanzetti) (1971),Drama
2503.0,5.0,"Apple, The (Sib) (1998)",Drama
557.0,5.0,Mamma Roma (1962),Drama
3607.0,5.0,One Little Indian (1973),Comedy|Drama|Western
